# Exploring Debugging Symbols

CFiddle can extract [DWARF](https://dwarfstd.org/doc/DWARF4.pdf) debugging information for compiled code.

This support is very rudimentary at the moment.  It's mostly suited for learning how DWARF works.

In [1]:
from cfiddle import *
configure_for_jupyter()

In [2]:
source = code(r"""
extern "C" 
int foo(int a) {
    int j= 1; 
    int k= 2; 
    for(register int i = 0; i < a; i++) {
        k =+i;
    }
    return j + k;
}""")
b = build(source)

  0%|          | 0/1 [00:00<?, ?it/s]

## Dump Debugging Information for a Compiled File

This can be very, very long.

In [3]:
print(b[0].debug_info())

[  11] DW_TAG_compile_unit
[  12]   DW_AT_producer = b'GNU C++11 9.3.0 -mtune=generic -march=x86-64 -g3 -std=gnu++11 -fPIC -fasynchronous-unwind-tables -fstack-protector-strong -fstack-clash-protection -fcf-protection'
[  16]   DW_AT_language = 4
[  17]   DW_AT_name = b'.cfiddle/builds/anonymous_code/b3509d8283a21391e6438dfd00197cc0.cpp'
[  21]   DW_AT_comp_dir = b'/cse142L/fiddle/examples'
[  25]   DW_AT_low_pc = 4345
[  33]   DW_AT_high_pc = 55
[  41]   DW_AT_stmt_list = 0
[  45]   DW_AT_GNU_macros = 0
[  49]   DW_TAG_subprogram
[  50]     DW_AT_external = True
[  50]     DW_AT_name = b'foo'
[  54]     DW_AT_decl_file = 1
[  55]     DW_AT_decl_line = 3
[  56]     DW_AT_decl_column = 5
[  57]     DW_AT_type = 153
[  61]     DW_AT_low_pc = 4345
[  69]     DW_AT_high_pc = 55
[  77]     DW_AT_frame_base = [156]
[  79]     DW_AT_GNU_all_call_sites = True
[  79]     DW_AT_sibling = 153
[  83]     DW_TAG_formal_parameter
[  84]       DW_AT_name = b'a'
[  86]       DW_AT_decl_file = 1
[  87]

## Or a Just a Function

In [4]:
print(b[0].debug_info("foo"))

[  49]   DW_TAG_subprogram
[  50]     DW_AT_external = True
[  50]     DW_AT_name = b'foo'
[  54]     DW_AT_decl_file = 1
[  55]     DW_AT_decl_line = 3
[  56]     DW_AT_decl_column = 5
[  57]     DW_AT_type = 153
[  61]     DW_AT_low_pc = 4345
[  69]     DW_AT_high_pc = 55
[  77]     DW_AT_frame_base = [156]
[  79]     DW_AT_GNU_all_call_sites = True
[  79]     DW_AT_sibling = 153
[  83]     DW_TAG_formal_parameter
[  84]       DW_AT_name = b'a'
[  86]       DW_AT_decl_file = 1
[  87]       DW_AT_decl_line = 3
[  88]       DW_AT_decl_column = 13
[  89]       DW_AT_type = 153
[  93]       DW_AT_location = [145, 84]
[  96]     DW_TAG_variable
[  97]       DW_AT_name = b'j'
[  99]       DW_AT_decl_file = 1
[ 100]       DW_AT_decl_line = 4
[ 101]       DW_AT_decl_column = 9
[ 102]       DW_AT_type = 153
[ 106]       DW_AT_location = [145, 100]
[ 109]     DW_TAG_variable
[ 110]       DW_AT_name = b'k'
[ 112]       DW_AT_decl_file = 1
[ 113]       DW_AT_decl_line = 5
[ 114]       DW_AT_decl

## Extracting Stack Frames

CFiddle has basic support for extracting the call frame for a function.  Which makes some sense if you look at it next to the source and the assembly.

The DWARF data tell us, for instance, that `a` is at offset -44 relative to the DWARF's concept of the base of the stack frame.  Simultaneously, the assembly suggests that `a` is  at offset -28 relative to `%rbp`, which is the architecture's notion of the base of the stack frame -- a difference of 16.  If you add 16 to the other offsets in stack frame output, the results line up with the assembly:  `j` is at -12, and `k` is at -16.  `i` is in a register.

Remarkably, DWARF4 includes a Turing-complete language for computing these and other values.  See, CFiddle taught you something already!

Obviously, there's more work to be done in in CFiddle to make the stack frames easier to understand.

In [6]:
from cfiddle.jupyter import compare
compare([b[0].stack_frame("foo"), b[0].source(), b[0].asm("foo")])